# **Library**

In [1]:
import numpy as np # numerical python

# **Arrays and Variables**

In [2]:
# arrays to test
A = np.array([[13, -3, 4],[1, 9, -5], [1, 1, 8]])
b = np.array([[-31],[-22],[-47]])

size = len(A) # size of A array

xsum = lsum = usum = 0 # auxiliares values to sum

# initial and cap error
erro = 1
eMax = 1E-2

# endline
ndln = '\n-------------------------------------------------------------------\n'

# **Gauss/LU**

*   *The Gauss method consists of change a linear system to an equivalent upper triangular linear system, as can be solved directly with back substitution.*
*   *The LU decomposition is that it provides an efficient way to compute the inverse matrix, and it provides a means of evaluating the conditioning of the system.*

In [3]:
U = np.concatenate((A, b), 1)
L = np.eye(size, size)
x = np.zeros(size)

for i in range(size):
  p = U[i, i]
  for j in range(i + 1, size):
    fator = U[j, i] / p
    L[j][i] = fator
    U[j,:] = U[j,:] - fator * U[i,:]

for i in range(size):
  for j in range(size):
    A[i][j] = U[i][j]
    b[i] = U[i, size]
    
for i in range(size - 1, -1, -1):
  sum = 0
  for j in range(size - 1, -1, -1):
    sum = sum + (A[i][j]) * (x[j])
  x[i] = ((b[i] - sum) / (A[i][i]))

for i in range(size):
  xsum = xsum + x[i]
  for j in range(size):
    usum = usum + U[i, j]
    lsum = lsum + L[i, j]

print(f'L:\n{L}',
      f'U:\n{U}',
      f'LU:\n{L@U}',
      f'L + U + x(n) = [{xsum+usum+lsum}]',
      sep = ndln)

L:
[[1.         0.         0.        ]
 [0.07692308 1.         0.        ]
 [0.07692308 0.11111111 1.        ]]
-------------------------------------------------------------------
U:
[[ 13  -3   4 -31]
 [  0   9  -5 -19]
 [  0   0   7 -41]]
-------------------------------------------------------------------
LU:
[[ 13.          -3.           4.         -31.        ]
 [  1.           8.76923077  -4.69230769 -21.38461538]
 [  1.           0.76923077   6.75213675 -45.4957265 ]]
-------------------------------------------------------------------
L + U + x(n) = [15.222222222222221]


# **Jacobi**

*   *The Jacobi method is a simplification of the Jacobi own values algorithm.*

In [4]:
K = np.zeros((size, size))
W = np.zeros((size))
y = np.zeros((size))

for i in range(0, size):
  K[i,:] = A[i,:] / A[i][i]
  W[i] = b[i] / A[i][i]
  K[i][i] = 0

K = -K

for it in range(1, 1000):
  d = y
  y = (K@y) + W
  erro = max(abs(y - d)) / max(abs(y))
  print(f'{it}th Interation: {y}\nerror: [{erro}]{ndln}')
  if erro < eMax: break


print(f'Answer check: {np.round(A@y, 0)}')

1th Interation: [-2.38461538 -2.11111111 -5.85714286]
error: [1.0]
-------------------------------------------------------------------

2th Interation: [-1.06959707 -5.36507937 -5.85714286]
error: [0.5555555555555557]
-------------------------------------------------------------------

3th Interation: [-1.82051282 -5.36507937 -5.85714286]
error: [0.12820512820512825]
-------------------------------------------------------------------

4th Interation: [-1.82051282 -5.36507937 -5.85714286]
error: [0.0]
-------------------------------------------------------------------

Answer check: [-31. -19. -41.]


# **Gauss Seidel**

*   *The Gauss Seidel method, also known as the method of successive displacement, is an iterative method used to solve a system of linear equations, and is similar to the Jacobi method.*

In [5]:
K = np.eye(size) - np.linalg.inv(np.eye(size)*A)@A
w = np.linalg.inv(np.eye(size)*A)@b
x = np.zeros((size, 1))

for i in range(size):
  xold = np.copy(x)
  for j in range(size):
    x[j, 0] = K[j,:]@x + w[j, 0]
    erro = np.max(np.abs(x-xold)) / np.max(np.abs(x))
  print(f'\n{i+1}th Interation\n',
        f'x:\n{x}',
        f'\nDifference between x in the {i+1}th and {i}th interaction:\n{x - xold}',
        f'\nerror: [{erro}]\n',
        sep = '\n', end = ndln)


1th Interation

x:
[[-2.38461538]
 [-2.11111111]
 [-5.85714286]]

Difference between x in the 1th and 0th interaction:
[[-2.38461538]
 [-2.11111111]
 [-5.85714286]]

error: [1.0]

-------------------------------------------------------------------

2th Interation

x:
[[-1.06959707]
 [-5.36507937]
 [-5.85714286]]

Difference between x in the 2th and 1th interaction:
[[ 1.31501832]
 [-3.25396825]
 [ 0.        ]]

error: [0.5555555555555557]

-------------------------------------------------------------------

3th Interation

x:
[[-1.82051282]
 [-5.36507937]
 [-5.85714286]]

Difference between x in the 3th and 2th interaction:
[[-0.75091575]
 [ 0.        ]
 [ 0.        ]]

error: [0.12820512820512825]

-------------------------------------------------------------------


# **Sassenfeld**

*   *The Sassenfeld number is another parameter used to verify the convergence of iterative methods.*

In [6]:
K = np.eye(size) - (np.linalg.inv(np.eye(size) * A) @ A)
W = (np.linalg.inv(np.eye(size)*A) @ b)
beta = np.ones((size, 1))
x = np.copy(b)

line = 0
colunm = 0

for i in range(size):
  x = K@x + W
  beta[i,0] = K[i,:] @ beta
  line = line + abs(K[:,i])
  colunm = colunm + abs(K[i,:])

sass = np.max(abs(beta))

print(f'K:\n{K}', f'Beta:\n{beta}', f'Sassenfeld: [{sass}]',
      f'Line sum: [{np.max(abs(line))}]', 
      f'Colunm sum: [{np.max(abs(colunm))}]', 
      sep = ndln)

K:
[[ 0.          0.23076923 -0.30769231]
 [ 0.          0.          0.55555556]
 [ 0.          0.          0.        ]]
-------------------------------------------------------------------
Beta:
[[-0.07692308]
 [ 0.55555556]
 [ 0.        ]]
-------------------------------------------------------------------
Sassenfeld: [0.5555555555555556]
-------------------------------------------------------------------
Line sum: [0.5555555555555556]
-------------------------------------------------------------------
Colunm sum: [0.8632478632478633]
